# Using IQP circuits to investigate n-node connected graphs

In [23]:
# Run this to import all the packages

import numpy as np
import networkx as nx
import math
import itertools
from sympy import fwht
import matplotlib.pyplot as plt
import pickle
import os
from joblib import Parallel, delayed
from collections import defaultdict, Counter
from matplotlib.gridspec import GridSpec
from itertools import combinations

## Define functions to generate and plot all n-node graphs

In [24]:
# Note: this is probably not the most efficent way to generate all non-isomorphic graphs but it only need to be run once
def generate_all_connected_graphs(n):
    nodes = list(range(n))
    all_possible_edges = list(combinations(nodes, 2))
    connected_graphs = []

    for i in range(1, 2 ** len(all_possible_edges)):
        edge_indices = [j for j in range(len(all_possible_edges)) if (i >> j) & 1]
        edges = [all_possible_edges[j] for j in edge_indices]
        G = nx.Graph()
        G.add_nodes_from(nodes)
        G.add_edges_from(edges)
        if nx.is_connected(G):
            if not any(nx.is_isomorphic(G, H) for H in connected_graphs):
                connected_graphs.append(G)
    return connected_graphs



def plot_graphs(graphs):
    n = len(graphs)
    cols = math.floor(n**0.5) + 1
    rows = (n // cols)
    fig, axes = plt.subplots(rows, cols, figsize=(3 * cols, 3 * rows))
    for i, ax in enumerate(axes.flat):
        if i < n:
            nx.draw(graphs[i], ax=ax, with_labels=True, node_color='lightblue', edge_color='gray')
            ax.set_title(f"Graph {i+1}")
        else:
            ax.axis('off')
    plt.tight_layout()
    plt.show()


## Plot all connected graphs for n nodes

## Set the number of nodes and save/load the set of connected graphs generated

In [25]:
# Set the number of nodes you want to look at
n_nodes = 7


# Create file directory
folder = "n node connected graphs"
os.makedirs(folder, exist_ok=True)
filename = os.path.join(folder, f"{n_nodes}-node connected graphs.pkl")


# Try to load from file
if os.path.exists(filename):
    print(f"Loading from file: {filename}")
    with open(filename, "rb") as f:
        connected_graphs_n = pickle.load(f)
else:
    # Save
    #generate data
    connected_graphs_n = generate_all_connected_graphs(n_nodes)
    with open(filename, "wb") as f:
        pickle.dump(connected_graphs_n, f)
    print(f"Saved to file: {filename}")



# Uncomment to plot all graphs
# print("Plotting data...")
# plot_graphs(connected_graphs_n)

Loading from file: n node connected graphs\7-node connected graphs.pkl


## Define fuctions that create the IQP circuit and simulate the distribution for each graph

In [26]:
# Compute the intermediate state before Hadamards
def compute_amplitude_vector(G, n, theta):
    state = np.zeros(2**n, dtype=complex)
    
    for x in range(2**n):
        bitstring = np.array(list(np.binary_repr(x, width=n)), dtype=int)
        phase = 0.0
        for (j, k) in G.edges:
            parity = (bitstring[j] + bitstring[k]) % 2
            phase += (-1)**parity
        amplitude = np.exp(1j * theta * phase) / np.sqrt(2**n)
        state[x] = amplitude

    return state

# Simulate the IQP circuit
def simulate_circuit(G, theta):
    n = len(G.nodes)
    intermediate_state = compute_amplitude_vector(G, n, theta)
    final_state = fwht(intermediate_state.copy()) / np.sqrt(len(intermediate_state))  # Apply FWHT and normalise
    probabilities = np.abs(final_state) ** 2
    return probabilities

# Generate distributions for each n node graph

### Function for generating the distributions for each graph over a range of theta values with save/load for easy access

In [27]:

# Simulation code with save/load
def generate_or_load_distributions(connected_graphs_n, n_nodes, thetas, filename=None):
    if filename is None:
        folder = "IQP distributions for n nodes"
        os.makedirs(folder, exist_ok=True)
        filename = os.path.join(folder, f"iqp_distributions_{n_nodes}nodes.pkl")
    if os.path.exists(filename):
        print(f"Loading distributions from {filename}...")
        with open(filename, 'rb') as f:
            data = pickle.load(f)
            return data['connected_graph_distributions'], data['thetas'], data['graph_labels']
    else:
        print("Generating distributions...")

        graph_labels = []
        connected_graph_distributions = {}

        # Simulate circuit for each theta value
        for i, G in enumerate(connected_graphs_n):
            graph_label = f"G_{i+1}"
            graph_labels.append(graph_label)
            connected_graph_distributions[graph_label] = [simulate_circuit(G, theta) for theta in thetas]

        # Save to file
        with open(filename, 'wb') as f:
            pickle.dump({
                'connected_graph_distributions': connected_graph_distributions,
                'thetas': thetas,
                'graph_labels': graph_labels,
                'n_nodes': n_nodes
            }, f)

        return connected_graph_distributions, thetas, graph_labels
    

### Generate data

In [28]:
# Define theta range
frames = 50
thetas = np.linspace(0, np.pi / 2, frames)

connected_graph_distributions, thetas, graph_labels = generate_or_load_distributions(connected_graphs_n, n_nodes, thetas)

Loading distributions from IQP distributions for n nodes\iqp_distributions_7nodes.pkl...


# Function for plotting bitstring proabilities

## This is used to compare and analyse distribtutions from specific graphs

In [29]:

#Use to look at specific graphs
def bit_reversed_indices(n):
    """Generate bit-reversed indices to spread colors apart."""
    width = int(np.ceil(np.log2(n)))
    # Reverse the binary representation of each index to create a new ordering
    return [int(f'{i:0{width}b}'[::-1], 2) for i in range(n)]


def plot_connected_graphs_bitstring_probabilities(
    connected_graph_distributions, 
    thetas, 
    n_nodes, 
    selected_graphs=None, 
    selected_bitstrings=None
):
    """
    Plot the probability of selected bitstrings for connected graphs as a function of θ.

    Parameters:
    
    connected_graph_distributions : dict
        Dictionary mapping graph labels (e.g., 'G_1') to their corresponding
        probability distributions for each bitstring.
        Each entry should be a 2D array of shape (len(thetas), 2**n_nodes).
    thetas : array-like
        Array of θ values used to generate the distributions.
    n_nodes : int
        Number of qubits/nodes in each graph.
    selected_graphs : list of str, optional
        List of graph labels to plot. Must match keys in connected_graph_distributions.
        If None, all graphs are plotted.
    selected_bitstrings : list of str, optional
        List of bitstrings to plot. If None, all bitstrings are plotted.

    
    Notes:
    
    - Graph labels must match keys in connected_graph_distributions, e.g. ['G_1', 'G_2'].
    - Uses bit_reversed_indices(n) to spread colors to make graphs more distinct when plotting.
    """
    all_bitstrings = [format(i, f'0{n_nodes}b') for i in range(2**n_nodes)]
    bitstring_indices = {bs: i for i, bs in enumerate(all_bitstrings)}

    # Use all graphs if none selected
    if selected_graphs is None:
        selected_graphs = list(connected_graph_distributions.keys())

    # Use all bitstrings if none selected
    if selected_bitstrings is None:
        selected_bitstrings = all_bitstrings

    # Make it a square layout
    n_bitstrings = len(selected_bitstrings)
    n_rows = int(np.ceil(np.sqrt(n_bitstrings)))
    n_cols = int(np.ceil(n_bitstrings / n_rows))

    # Create subplot grid
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 15), sharex=True, sharey=True)
    axes = axes.flatten()

    # Make graphs that are close together in number have distinct colours
    cmap = plt.colormaps['hsv']
    indices = bit_reversed_indices(len(selected_graphs))
    colors = [cmap(i / len(selected_graphs)) for i in indices]

    # Plot probability of each selected bitstring for each selected graph
    for idx, graph_label in enumerate(selected_graphs):
        probs = np.array(connected_graph_distributions[graph_label])
        for i, bitstring in enumerate(selected_bitstrings):
            bit_index = bitstring_indices[bitstring]
            label = graph_label if i == 0 else None
            axes[i].plot(thetas / np.pi, probs[:, bit_index], label=label, color=colors[idx])

    # Set titles, labels, and limits for each subplot
    for i, bitstring in enumerate(selected_bitstrings):
        axes[i].set_title(bitstring)
        axes[i].set_ylim(0, 1)
        axes[i].set_xlabel(r'$2\theta / \pi$')
        axes[i].set_ylabel('Probability')

    # Hide unused subplots
    for j in range(n_bitstrings, len(axes)):
        axes[j].axis('off')

    # Add overall figure title and legend
    fig.suptitle(rf"{n_nodes}-Qubit Selected Bitstrings vs $\theta$", fontsize=20)
    fig.legend(loc="center right")
    fig.tight_layout(rect=[0, 0, 0.95, 0.95])
    plt.show()



# Plot all bitstring probabilities

In [30]:
# Uncomment to plot

#plot_connected_graphs_bitstring_probabilities(connected_graph_distributions, thetas, n_nodes, selected_graphs=None, selected_bitstrings=None)

# Example comparing G_1 and G_2
#plot_connected_graphs_bitstring_probabilities(connected_graph_distributions, thetas, n_nodes, selected_graphs=['G_1','G_2'], selected_bitstrings=None)

# Comparing the graphs using different metrics

### Define functions to compute each metric and generate the relevant similarity matrix

In [31]:
# Similarity/distance/fidelitiy functions
def similarity(p, q):
    p = np.asarray(p, dtype=np.float64)
    q = np.asarray(q, dtype=np.float64)
    return (np.inner(p,q)/(np.linalg.norm(p)*np.linalg.norm(q)))

def tvd(p, q):
    return 0.5 * np.sum(np.abs(p - q))

def fidelity(p, q):
    p = np.asarray(p, dtype=np.float64)
    q = np.asarray(q, dtype=np.float64)
    return (np.sum(np.sqrt(p * q)))**2


# Generate similarity matrices
def compute_similarity_for_graph(g1_label, metric_func, graph_labels, thetas, connected_graph_distributions):
    sim_matrix = np.zeros((len(graph_labels), len(thetas)))
    for i, theta in enumerate(thetas):
        p1 = connected_graph_distributions[g1_label][i]
        for j, g2_label in enumerate(graph_labels):
            p2 = connected_graph_distributions[g2_label][i]
            sim_matrix[j, i] = metric_func(p1, p2)
    return g1_label, sim_matrix

# Function for doing this in parallel to speed up compute time.
# This is especially needed for n_nodes > 5 due to the number of graphs.
def compute_similarity_matrix_parallel(metric_func):
    print('Computing similarity data in parallel...')
    return dict(
        Parallel(n_jobs=-1)(
            delayed(compute_similarity_for_graph)(g1_label, metric_func, graph_labels, thetas, connected_graph_distributions)
            for g1_label in graph_labels
        )
    )


### Generate date for your selected metric and save/load from file for easy access

In [32]:
# Select metric
metric = tvd # or similariy or fidelity

folder = "n node similarity data"
os.makedirs(folder, exist_ok=True)
metric_name = metric.__name__
filename = os.path.join(folder, f"{n_nodes}_node_similarity_data_metric_{metric_name}.pkl")
if os.path.exists(filename):
    print(f"Loading similarity data from: {filename}")
    with open(filename, "rb") as f:
        similarity_data = pickle.load(f)
else:
    similarity_data = compute_similarity_matrix_parallel(metric)
    print(f"Saving similarity data for metric function = {metric_name}...")
    with open(filename, "wb") as f:
        pickle.dump(similarity_data, f)

Loading similarity data from: n node similarity data\7_node_similarity_data_metric_tvd.pkl


### Plotting

In [33]:
def plot_similarity_data(similarity_data):
    # Configure plot
    batch_size = 9                           # Number of graphs to show per batch of subplots
    n_graphs = len(graph_labels)            
    n_batches = math.ceil(n_graphs / batch_size)  # Number of batches needed to show all graphs

    # Loop over each batch of graphs
    for batch_idx in range(n_batches):
        # Selecting the subset of graph labels for this batch
        start = batch_idx * batch_size
        end = min(start + batch_size, n_graphs) # This min is needed so that the last page formats correctly
        batch_labels = graph_labels[start:end] # Slice out of the list of graph labels
        
        # Setup figure size
        n_subplots = len(batch_labels)       
        cols = 3                             
        rows = math.ceil(n_subplots / cols) 
        fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 4 * rows))

        axes = axes.flatten() # For easy iteration

        # Loop over each subplot position and corresponding graph label
        for ax, g1_label in zip(axes, batch_labels):
            sim_matrix = similarity_data[g1_label]

            # Display similarity matrix as heatmap
            heatmap = ax.imshow(
                sim_matrix,
                aspect='auto',                # Stretch to fill subplot space
                origin='lower',               # Sets G_0 to be the bottom graph
                cmap='magma_r',               
                extent=[0, 1, 0, len(graph_labels)], 
                vmin=0, vmax=1                
            )

            # Set subplot title and axis labels
            ax.set_title(g1_label, fontsize=12) 
            ax.set_yticks(np.arange(len(graph_labels)) + 0.5)  # Center the y labels
            ax.set_yticklabels(graph_labels, fontsize=8)       # Show all graph labels up y axis
            ax.set_xticks([0, 0.25, 0.5, 0.75, 1])            
            ax.set_xticklabels(                               # LaTeX labels for x axis
                [r'$0$', r'$\frac{\pi}{4}$', r'$\frac{\pi}{2}$',
                r'$\frac{3\pi}{4}$', r'$\pi$'],
                fontsize=10
            )
            ax.set_xlabel(r'$\theta$', fontsize=12)
            ax.set_ylabel('Graphs', fontsize=12)

            # Add an individual colorbar for this subplot
            fig.colorbar(heatmap, ax=ax, orientation='vertical', pad=0.01, fraction=0.05)

        # Hide any unused subplot axes (when batch has < rows*cols subplots)
        for i in range(n_subplots, len(axes)):
            axes[i].axis('off')

        # Add a figure-wide title for the batch
        fig.suptitle(
            f'{metric.__name__} of probability distributions between {n_nodes}-vertex connected graphs\n'
            f'Batch {batch_idx + 1}/{n_batches}',
            fontsize=16
        )

        # Adjust layout so titles/labels don’t overlap and leave space for page title
        plt.tight_layout(rect=[0, 0, 1, 0.96])

        plt.show()

In [34]:
# Uncomment to plot data:

#plot_similarity_data(similarity_data)

# Creating custom graphs for checking isomorphisms

### Function for making custom permutations of graphs

In [35]:
def permute_graph_nodes(G, permutation_map):
    """
    Returns a new graph with nodes relabeled according to the given permutation map.

    Parameters:
    G: networkx.Graph 
        The selected graph to permute
    permutation_map: dict 
        Mapping from old node labels to new ones

    Returns:
    networkx.Graph 
        The permuted graph
    """
    return nx.relabel_nodes(G, permutation_map, copy=True)


# Custom permutations - some examples. Feel free to create your own.
cyclic_map_1 = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5, 5: 6, 6: 0} # Note: these two only work for n_nodes = 7
cyclic_map_2 = {0: 2, 1: 3, 2: 4, 3: 5, 4: 6, 5: 0, 6: 1} # Change the indices to go up to n_nodes - 1 for this to work properly
c_perm_map_1 = {0: 2, 2: 4, 4: 0}
c_perm_map_2 = {1: 4, 4: 3, 3: 1}


### Function to generate all permutations for n nodes

In [36]:
def generate_all_permutation_maps(n_nodes):
    """
    Generates a list of all unique node relabeling permutations for a graph with n_nodes. 
    Stores each permutation as a dictionary for use in permute_graph_nodes(G, permutation_map).
    
    Parameters:
    n_nodes: int 
        Number of nodes in the graph.
    
    Returns:
    perm_maps: list of dict 
        List of all permutation maps.
    """
    node_labels = list(range(n_nodes))
    perm_maps = []

    for i, perm in enumerate(itertools.permutations(node_labels)):
        perm_map = {old: new for old, new in zip(node_labels, perm)}
        perm_maps.append(perm_map)

    print(f"{len(perm_maps)} permutations generated and saved as perm_maps[0] to perm_maps[{len(perm_maps)-1}]")
    return perm_maps

In [37]:
all_permutations = generate_all_permutation_maps(n_nodes)

5040 permutations generated and saved as perm_maps[0] to perm_maps[5039]


In [38]:
# Define your custom graphs
custom_graphs = [
    *[permute_graph_nodes(connected_graphs_n[2], perm) for perm in all_permutations],
    *[permute_graph_nodes(connected_graphs_n[3], perm) for perm in all_permutations],
    permute_graph_nodes(connected_graphs_n[4],c_perm_map_2)
]
custom_labels = [
    *[f"G3 with permutation #{i}: {all_permutations[i]}" for i in range(len(all_permutations))],
    *[f"G4 with permutation #{i}: {all_permutations[i]}" for i in range(len(all_permutations))],
    f"G5 with permutation {c_perm_map_2}"
]

## Function for finding identical distributions at custom theta values

In [39]:
def find_identical_distributions_at_theta_values(
    connected_graphs_n = connected_graphs_n,
    graph_labels = graph_labels,
    theta_values = [np.pi/3, np.pi/6], 
    metric = tvd,
    similarity_tolerance=0.02,
    identity_tolerance=1e-5,
    custom_graphs=None,
    custom_labels=None
):
    """
    Simulates IQP circuit distributions for all graphs at specified theta values
    and compares them pairwise using total variation distance.

    - Graph pairs with distance < identity_tolerance are classified as "identical".
    - Graph pairs with distance < similarity_tolerance but > identity_tolerance
      are classified as "similar".
    - Graph pairs with distance >= similarity_tolerance are classified as "different".
    - Special handling is applied for graphs whose labels indicate permutations.

    Parameters:
    connected_graphs_n : list of nx.Graph
        List of connected graphs with n nodes.
    target_theta_values : list of float
        Theta values at which to simulate and compare distributions (default: [np.pi/3, np.pi/6]).
    metric : callable
        A function to compute the distance between two distributions (default: tvd).
        (Expected signature: metric(dist1, dist2) -> float).
    graph_labels : list of str
        Labels for the graphs in connected_graphs_n.
    similarity_tolerance : float, optional
        Threshold below which graphs are considered "similar" (default: 0.02).
    identity_tolerance : float, optional
        Threshold below which graphs are considered "identical" (default: 1e-5).
    custom_graphs : list of nx.Graph, optional
        Additional user-supplied graphs to include in the comparison.
    custom_labels : list of str, optional
        Labels for custom_graphs. If None, defaults to "custom_i".

    Returns:
    identical : list of tuple
        Pairs of graphs that are identical (indices, metric comparison result).
    similar : list of tuple
        Pairs of graphs that are similar but not identical (indices, metric comparison result).
    different : list of tuple
        Pairs of graphs that are distinguishable (indices, metric comparison result).
    permutation_matches : list of tuple
        Pairs of graphs marked as identical but one is a permutation.
    """

    # Combine connected and custom graphs
    if custom_graphs is None:
        all_graphs = connected_graphs_n
        all_labels = graph_labels
    else:
        all_graphs = custom_graphs + connected_graphs_n
        all_labels = (custom_labels if custom_labels else [f"custom_{i}" for i in range(len(custom_graphs))]) + graph_labels

    num_graphs = len(all_graphs)

    for theta in theta_values:
        print(f"\nSimulating all graphs at theta = {theta}...")

        # Simulate probability distributions for each graph at this theta
        theta_distributions = [
            np.array(simulate_circuit(G, theta), dtype=float)
            for G in all_graphs
        ]

        # Containers for classification results
        identical = []
        similar = []
        different = []
        permutation_matches = []

        # Pairwise comparisons between all graph distributions
        for i, j in combinations(range(num_graphs), 2):
            d1 = theta_distributions[i]
            d2 = theta_distributions[j]
            metric_comparison = metric(d1, d2)

            label_i = all_labels[i]
            label_j = all_labels[j]

            if metric_comparison < identity_tolerance:
                # Treat permutation-labeled graphs separately
                if ("with permutation" in label_i) or ("with permutation" in label_j):
                    permutation_matches.append((i, j, metric_comparison))
                else:
                    identical.append((i, j, metric_comparison))
            elif metric_comparison < similarity_tolerance:
                similar.append((i, j, metric_comparison))
            else:
                different.append((i, j, metric_comparison))

        # Printing results
        print(f"\nResults for θ = {theta}:")

        if identical:
            for i, j, d in identical:
                print(f" {all_labels[i]} ≡ {all_labels[j]} (TVD = {d:.2e})")
            print(f"{len(identical)} Identical, non-permuted graph pairs (Distance < {identity_tolerance})")
        else:
            print("No identical pairs.")

        print(f"{len(permutation_matches)} Identical G1-permutation matches")

        if similar:
            print(f"{len(similar)} Similar graph pairs (Distance < {similarity_tolerance}):")
            for i, j, d in similar:
                print(f"  {all_labels[i]} ~ {all_labels[j]}  (TVD = {d:.2e})")
        else:
            print(f"No similar pairs (i.e., pairs with nonzero distance < {similarity_tolerance}).")

        if not identical and not similar:
            print(f"All graphs are distinguishable for this theta within a tolerance of {similarity_tolerance}.")

        # Visualization of identical graph pairs
        n_nodes = all_graphs[0].number_of_nodes()
        bitstrings = [format(k, f'0{n_nodes}b') for k in range(2**n_nodes)]

        for i, j, _ in identical:
            fig, axes = plt.subplots(1, 3, figsize=(12, 4))

            nx.draw(all_graphs[i], ax=axes[0], with_labels=True, node_color='lightblue')
            axes[0].set_title(f"Graph {all_labels[i]}")

            nx.draw(all_graphs[j], ax=axes[1], with_labels=True, node_color='lightgreen')
            axes[1].set_title(f"Graph {all_labels[j]}")

            axes[2].bar(bitstrings, theta_distributions[i])
            axes[2].set_title("Identical Distribution")
            axes[2].set_xlabel("Bitstring")
            axes[2].set_ylabel("Probability")
            axes[2].tick_params(axis='x', labelrotation=90)

            plt.suptitle(f"Identical graphs at theta = {theta}")
            plt.tight_layout()
            plt.show()


In [40]:
# Compare graphs and their isomorphisms using tvd 

# Uncomment to run
#find_identical_distributions_at_theta_values(custom_graphs=custom_graphs,custom_labels=custom_labels)

# Function for checking the hamming weight and multiplicities for graphs

In [41]:
def compare_graphs_by_amplitudes_and_hamming_weight(
    custom_graphs=None,
    custom_labels=None,
    selected_indices=None,
    graph_labels=graph_labels,
    connected_graphs_n=connected_graphs_n,
    theta_values=[np.pi/3, np.pi/6],
    n_nodes=n_nodes,
    compare_all=False,
    tolerance = 0.002
):
    """
    Simulates IQP distributions for thetas in theta_values on selected or all graphs,
    checks Hamming weight distributions and amplitude multiplicities.
    Checks matches for each theta value and then for all theta values.

    - If multiplicities differ => graphs are definitely non-isomorphic.
    - If multiplicities match => graphs could be isomorphic.

    Parameters:

    custom_graphs : list of nx.Graph
        Custom graphs to include.
    custom_labels : list of str
        Labels for custom graphs.
    selected_indices : list of int
        Indices of graphs from connected_graphs_n to include (ignored if compare_all=True).
    graph_labels : list of str
        Labels for connected_graphs_n.
    connected_graphs_n : list of nx.Graph
        Set of connected graphs with n nodes.
    theta_values : list of numpy expressions
        Theta values to test (default: [pi/3, pi/6]).
    n_nodes : int
        Number of nodes in the graphs.
    compare_all : bool
        If True, compares all graphs in connected_graphs_n + custom_graphs.
    tolerance : float
        The tolerance for a probability value to count as non-zero
    """

    results = {}

    for theta in theta_values:

        # Collect graphs and labels
        all_graphs, all_labels = [], []

        if custom_graphs:
            all_graphs.extend(custom_graphs)
            all_labels.extend(custom_labels if custom_labels else [f"custom_{i}" for i in range(len(custom_graphs))])

        if connected_graphs_n and graph_labels:
            if compare_all:
                all_graphs.extend(connected_graphs_n)
                all_labels.extend(graph_labels)
            elif selected_indices:
                for idx in selected_indices:
                    all_graphs.append(connected_graphs_n[idx])
                    all_labels.append(graph_labels[idx])

        if not all_graphs:
            raise ValueError("No graphs selected.")

        # Simulate distributions
        theta_distributions = [
            np.array(simulate_circuit(G, theta), dtype=float)
            for G in all_graphs
        ]

        # Hamming weight distribution + multiplicities

        bitstrings = [format(k, f'0{n_nodes}b') for k in range(2**n_nodes)]
        hamming_weights = [bs.count("1") for bs in bitstrings]
        summary = {}
        for label, dist in zip(all_labels, theta_distributions):
            hw_dict = defaultdict(list)
            for bs, hw, p in zip(bitstrings, hamming_weights, dist):
                if p > tolerance:  # ignore values smaller than the tolerace
                    hw_dict[hw].append(round(p, 10))  # round to remove noise

            multiplicities = {hw: Counter(vals) for hw, vals in hw_dict.items()}
            summary[label] = {
                "hamming_weight_distribution": {hw: sum(hw_dict[hw]) for hw in hw_dict},
                "multiplicities": multiplicities
            }

        # Compare graphs by multiplicities
        print(f"\nMultiplicity Comparison at theta = {theta}")
        labels = list(summary.keys())
        matching_pairs = []
        for i in range(len(labels)):
            for j in range(i+1, len(labels)):
                mult1 = summary[labels[i]]["multiplicities"]
                mult2 = summary[labels[j]]["multiplicities"]
                if mult1 == mult2:
                    print(f"{labels[i]} vs {labels[j]}: multiplicities match => could be isomorphic")
                    matching_pairs.append((labels[i], labels[j]))

        results[theta] = {
            "summary": summary,
            "matching_pairs": set(matching_pairs)  # store as set for easy intersection later
        }

    # Find pairs that matched across all thetas
    if results:
        common_pairs = set.intersection(*(results[theta]["matching_pairs"] for theta in theta_values))
        print("\nGraphs that are isomorphic across all theta values:")
        if common_pairs:
            for pair in common_pairs:
                print(f"{pair[0]} vs {pair[1]}")
        else:
            print("No graph pairs matched across all theta values.")

    #return results

In [42]:
compare_graphs_by_amplitudes_and_hamming_weight(compare_all=True)


Multiplicity Comparison at theta = 1.0471975511965976
G_43 vs G_91: multiplicities match => could be isomorphic
G_108 vs G_110: multiplicities match => could be isomorphic
G_129 vs G_142: multiplicities match => could be isomorphic
G_151 vs G_234: multiplicities match => could be isomorphic
G_313 vs G_363: multiplicities match => could be isomorphic
G_330 vs G_368: multiplicities match => could be isomorphic
G_344 vs G_501: multiplicities match => could be isomorphic
G_382 vs G_418: multiplicities match => could be isomorphic

Multiplicity Comparison at theta = 0.5235987755982988

Graphs that are isomorphic across all theta values:
No graph pairs matched across all theta values.


### Version of the function used for looking at specific graphs with plotting functionality

In [43]:
#Plotting function
def compare_plot_graphs_by_amplitudes_and_hamming_weight(
    custom_graphs=None,
    custom_labels=None,
    selected_indices=None,
    graph_labels=graph_labels,
    connected_graphs_n=connected_graphs_n,
    theta_values=[np.pi/3, np.pi/6],
    n_nodes=n_nodes
):
    """
    Simulates IQP distributions for thetas in theta_values on selected or all graphs,
    checks Hamming weight distributions and amplitude multiplicities.
    Checks matches for each theta value.
    Plots the network x graphs, their distributions at each theta value and their Hamming weights.

    - If multiplicities differ => graphs are definitely non-isomorphic.
    - If multiplicities match => graphs could be isomorphic.

    Parameters:

    custom_graphs : list of nx.Graph
        Custom graphs to include.
    custom_labels : list of str
        Labels for custom graphs.
    selected_indices : list of int
        Indices of graphs from connected_graphs_n to include (ignored if compare_all=True).
    graph_labels : list of str
        Labels for connected_graphs_n.
    connected_graphs_n : list of nx.Graph
        Set of connected graphs with n nodes.
    theta_values : list of numpy expressions
        Theta values to test (default: [pi/3, pi/6]).
    n_nodes : int
        Number of nodes in the graphs.
    """

    for theta in theta_values:

        # --- Collect graphs and labels
        all_graphs, all_labels = [], []
        if custom_graphs:
            all_graphs.extend(custom_graphs)
            all_labels.extend(custom_labels if custom_labels else [f"custom_{i}" for i in range(len(custom_graphs))])
        if selected_indices and connected_graphs_n and graph_labels:
            for idx in selected_indices:
                all_graphs.append(connected_graphs_n[idx])
                all_labels.append(graph_labels[idx])
        if not all_graphs:
            raise ValueError("No graphs selected.")

        
        # --- Simulate distributions
        raw_distributions = [
            np.array(simulate_circuit(G, theta), dtype=float)
            for G in all_graphs
        ]

        bitstrings = [format(k, f'0{n_nodes}b') for k in range(2**n_nodes)]
        hamming_weights = [bs.count("1") for bs in bitstrings]

        # Hamming weight distribution + multiplicities
        summary = {}
        for label, dist in zip(all_labels, raw_distributions):
            hw_dict = defaultdict(list)
            for bs, hw, p in zip(bitstrings, hamming_weights, dist):
                if p > 1e-12:  # ignore exact zeros
                    hw_dict[hw].append(round(p, 10))  # round to remove noise

            multiplicities = {hw: Counter(vals) for hw, vals in hw_dict.items()}
            summary[label] = {
                "hamming_weight_distribution": {hw: sum(hw_dict[hw]) for hw in hw_dict},
                "multiplicities": multiplicities
            }

        # Check isomorphism candidates
        print("=== Multiplicity Comparison ===")
        labels = list(summary.keys())
        for i in range(len(labels)):
            for j in range(i+1, len(labels)):
                mult1 = summary[labels[i]]["multiplicities"]
                mult2 = summary[labels[j]]["multiplicities"]
                if mult1 == mult2:
                    print(f"{labels[i]} vs {labels[j]}: multiplicities match => could be isomorphic")
                else:
                    print(f"{labels[i]} vs {labels[j]}: multiplicities differ => non-isomorphic")

        # Plotting
        num_graphs = len(all_graphs)
        fig = plt.figure(figsize=(max(20, 3*num_graphs), 12))
        gs = GridSpec(3, num_graphs, height_ratios=[1, 1.5, 1], hspace=0.6, wspace=0.4)

        # Top row: nx graph plots
        for i, (graph, label) in enumerate(zip(all_graphs, all_labels)):
            ax_graph = fig.add_subplot(gs[0, i])
            pos = nx.spring_layout(graph, seed=42) if len(graph.nodes()) <= 6 else nx.circular_layout(graph)
            nx.draw(graph, pos, ax=ax_graph,
                    with_labels=True, node_color="lightblue", node_size=400,
                    font_size=8, font_weight="bold", edge_color="gray")
            ax_graph.set_title(label, fontsize=8, fontweight="bold")
            ax_graph.axis("off")

        # Middle row: probability distributions
        ax_bar = fig.add_subplot(gs[1, :])
        x = np.arange(len(bitstrings))
        width = 0.8 / len(all_graphs)

        # Make graphs have distinct colours
        cmap = plt.colormaps['hsv']
        indices = bit_reversed_indices(len(all_graphs))
        colors = [cmap(i / len(all_graphs)) for i in indices]

        for i, (dist, label) in enumerate(zip(raw_distributions, all_labels)):
            ax_bar.bar(x + i*width, dist, width=width, label=label, color=colors[i])
        ax_bar.set_xticks(x + width*(len(all_graphs)-1)/2)
        ax_bar.set_xticklabels(bitstrings, rotation=90)
        ax_bar.set_ylabel("Probability")
        ax_bar.set_xlabel("Bitstring")
        ax_bar.set_title(f"Graph distributions at theta = {theta}")
        ax_bar.legend(loc="upper right", fontsize="small")

        # Bottom row: Hamming weight summary
        for i, label in enumerate(all_labels):
            ax_hw = fig.add_subplot(gs[2, i])
            hw_dist = summary[label]["hamming_weight_distribution"]
            ax_hw.bar(hw_dist.keys(), hw_dist.values(), color=colors[i])
            ax_hw.set_title(f"Hamming Weights\n{label}", fontsize=8)
            ax_hw.set_xlabel("Hamming Weight")
            ax_hw.set_ylabel("Total Prob.")

        plt.show()

In [44]:
# Uncomment to plot

#compare_plot_graphs_by_amplitudes_and_hamming_weight(selected_indices=[42,90,107,109])
#compare_plot_graphs_by_amplitudes_and_hamming_weight(selected_indices=[128,141,150,233])
#compare_plot_graphs_by_amplitudes_and_hamming_weight(selected_indices=[312,362,329,367])
#compare_plot_graphs_by_amplitudes_and_hamming_weight(selected_indices=[343,500,381,417])
#compare_plot_graphs_by_amplitudes_and_hamming_weight(selected_indices=[153,265])
#compare_plot_graphs_by_amplitudes_and_hamming_weight(selected_indices=[13,17])